##### **Bibliotecas**

In [ ]:
import os
import numpy as np
import pandas as pd
from joblib import dump, load
from openai import OpenAI, AsyncOpenAI
import nest_asyncio
import asyncio
nest_asyncio.apply()
from warnings import filterwarnings
filterwarnings('ignore')

##### **Dataset**

In [3]:
url = os.getenv("URL_DATASET")
df = pd.read_excel(url)
df.head(3)

,Text,Text_PT,Opinion(1/0),Question(1/0),Answer(1/0),Sentiment(1-7),Confusion(1-7),Urgency(1-7),CourseType,forum_post_id,course_display_name,forum_uid,created_at,post_type,anonymous,anonymous_to_peers,up_count,comment_thread_id,reads
0,Interesting! How often we say those things to ...,Interessante! Quantas vezes dizemos essas cois...,1,0,0,6.5,2.0,1.5,Education,5225177f2c501f0a00000015,Education/EDUC115N/How_to_Learn_Math,30CADB93E6DE4711193D7BD05F2AE95C,2013-09-02 22:55:59,Comment,0.0,0.0,0.0,5221a8262cfae31200000001,41.0
1,"What is \Algebra as a Math Game\"" or are you j...",O que é \ Álgebra como um jogo de matemática \...,0,1,0,4.0,5.0,3.5,Education,5207d0e9935dfc0e0000005e,Education/EDUC115N/How_to_Learn_Math,37D8FAEE7D0B94B6CFC57D98FD3D0BA5,2013-08-11 17:59:05,Comment,0.0,0.0,0.0,520663839df35b0a00000043,55.0
2,I like the idea of my kids principal who says ...,Gosto da ideia do diretor dos meus filhos que ...,1,0,0,5.5,3.0,2.5,Education,52052c82d01fec0a00000071,Education/EDUC115N/How_to_Learn_Math,CC11480215042B3EB6E5905EAB13B733,2013-08-09 17:53:06,Comment,0.0,0.0,0.0,51e59415e339d716000001a6,25.0


In [9]:
dump(df, "data/df_stanford.z")

['data/df_stanford.z']

##### **Dataframe salvo**

In [10]:
df = load('data/df_stanford.z')
df.head(3)

,Text,Text_PT,Opinion(1/0),Question(1/0),Answer(1/0),Sentiment(1-7),Confusion(1-7),Urgency(1-7),CourseType,forum_post_id,course_display_name,forum_uid,created_at,post_type,anonymous,anonymous_to_peers,up_count,comment_thread_id,reads
0,Interesting! How often we say those things to ...,Interessante! Quantas vezes dizemos essas cois...,1,0,0,6.5,2.0,1.5,Education,5225177f2c501f0a00000015,Education/EDUC115N/How_to_Learn_Math,30CADB93E6DE4711193D7BD05F2AE95C,2013-09-02 22:55:59,Comment,0.0,0.0,0.0,5221a8262cfae31200000001,41.0
1,"What is \Algebra as a Math Game\"" or are you j...",O que é \ Álgebra como um jogo de matemática \...,0,1,0,4.0,5.0,3.5,Education,5207d0e9935dfc0e0000005e,Education/EDUC115N/How_to_Learn_Math,37D8FAEE7D0B94B6CFC57D98FD3D0BA5,2013-08-11 17:59:05,Comment,0.0,0.0,0.0,520663839df35b0a00000043,55.0
2,I like the idea of my kids principal who says ...,Gosto da ideia do diretor dos meus filhos que ...,1,0,0,5.5,3.0,2.5,Education,52052c82d01fec0a00000071,Education/EDUC115N/How_to_Learn_Math,CC11480215042B3EB6E5905EAB13B733,2013-08-09 17:53:06,Comment,0.0,0.0,0.0,51e59415e339d716000001a6,25.0


##### **Rótulos de Sentimentos**

In [11]:
np.sort(df['Sentiment(1-7)'].unique())

array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5, 7. ])

##### **Rotulando post com GPT**

In [21]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def gpt_label(text):
    try:
        response = client.chat.completions.create(
            # model="gpt-3.5-turbo",
            # model="gpt-4o",
            model="gpt-4o-mini",
            max_tokens=3,
            store=True,
            messages=[
                {
                    "role": "user",
                    "content": f"You are a data scientist who labels texts regarding sentiment with one of the options [1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5, 7. ], with 1 being very negative sentiment and up to 7 being very positive sentiment. Answer only one number and label the following text: {text}",
                },
            ],
        )
        return response.choices[0].message
    except Exception as e:
        print(f'Erro ao rotular: {str(e)}')
        return 0

In [22]:
indice_escolhido = 2
rotulo = float(gpt_label(df["Text"][indice_escolhido]).content)
print(f'Rótulo escolhido pelo GPT: {rotulo}')
print('-'*50)
print('Post:')
print(df[["Text", "Sentiment(1-7)"]].iloc[indice_escolhido]['Text'])
print('Rótulo real: ', df[["Text", "Sentiment(1-7)"]].iloc[indice_escolhido]["Sentiment(1-7)"])

Rótulo escolhido pelo GPT: 5.5
--------------------------------------------------
Post:
I like the idea of my kids principal who says \Smart doesn't mean easy, smart means working hard\" and incorporating the idea of making mistakes into working hard."
Rótulo real:  5.5


In [125]:
df[["Text", "Sentiment(1-7)"]]

,Text,Sentiment(1-7)
0,Interesting! How often we say those things to ...,6.5
1,"What is \Algebra as a Math Game\"" or are you j...",4.0
2,I like the idea of my kids principal who says ...,5.5
3,"From their responses, it seems the students re...",6.0
4,"The boys loved math, because \there is freedom...",7.0
...,...,...
29599,The p value tells us the probability of observ...,4.0
29600,given the null hypothesis is considered true,4.0
29601,"> Hello Josh,_x0007__x0007_Is this hypothesis ...",4.0
29602,"Hi Josh,_x0007__x0007__x0007__x0007_Looking at...",3.5


##### **Rotulando dataset completo com GPT**

In [23]:
client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

async def gpt_label_async(text):
    try:
        response = await client.chat.completions.create(    
            # model="gpt-3.5-turbo",
            # model="gpt-4o",
            model="gpt-4o-mini",
            max_tokens=3,
            store=True,
            messages=[
                {
                    "role": "user",
                    "content": f"You are a data scientist who labels texts regarding sentiment with one of the options [1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5, 7. ], with 1 being very negative sentiment and up to 7 being very positive sentiment. Answer only one number and label the following text: {text}",
                },
            ],
        )
        return response.choices[0].message
    except Exception as e:
        print(f'Erro ao rotular: {str(e)}')
        return 0

In [ ]:
rotulos = df["Text"][0:20].apply(lambda x: asyncio.run(gpt_label_async(x)))
rotulos = rotulos.map(lambda x: float(x.content))

In [117]:
rotulos

0     5.0
1     3.0
2     5.0
3     6.0
4     5.0
5     2.0
6     3.0
7     4.0
8     6.0
9     4.0
10    6.0
11    6.0
12    2.0
13    5.0
14    2.0
15    5.0
16    6.0
17    5.0
18    6.0
19    3.0
Name: Text, dtype: float64